# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'up-front-server-capex'
compute_percentile_threshold = 50  # select systems >= this percentile of training compute
variant = 'original'

if estimation_method == 'amortized':
    cost_estimation_function = estimate_amortized_hardware_costs
elif estimation_method == 'up-front-server-capex':
    cost_estimation_function = estimate_upfront_server_capex
else:
    cost_estimation_function = estimate_costs

results_dir = f'results/{estimation_method}-{compute_percentile_threshold}th-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_percentile_threshold=compute_percentile_threshold
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(148, 5510, 142)

# Cost estimation

In [6]:
cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

Estimating up-front server capex
==== System: Gemini Ultra ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: Qwen-72B ====
Could not find hardware model for Qwen-72B


==== System: Inflection-2 ====
Trying NVIDIA H100 SXM5
Found price: 39546.976

==== System: Nemotron-3-8B ====
Trying NVIDIA A100
Soft matching NVIDIA A100 to NVIDIA A100
Soft matching NVIDIA A100 to NVIDIA A100 PCIe
Found price: 13750.0

==== System: Yi-34B ====
Could not find hardware model for Yi-34B


==== System: Skywork-13B ====
Trying NVIDIA A800
Could not find hardware model for NVIDIA A800


==== System: ChatGLM3 ====
Could not find hardware model for ChatGLM3


==== System: XGen-7B ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: Falcon 180B ====
Trying NVIDIA A100 SXM4 40 GB
Found price: 17458.333333333332

==== System: Llama 2-70B ====
Trying NVIDIA A100 SXM4 80 GB
Found price: 25641.25

==== System: Llama 2-34B ====
Trying NVIDIA 

In [7]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Cost
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,197734880.0
59,Nemotron-3-8B,Language,"Chat,Language generation",NaN,SOTA improvement,"""The Nemotron-3-8B-QA model offers state-of-th...",Permissive license (depr.),https://developer.nvidia.com/blog/nvidia-ai-fo...,NaN,NVIDIA AI Foundation Models: Build Custom Ente...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14080000.0
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN,241440.0
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,154061.0,Deep Residual Learning for Image Recognition,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN,NaN


In [8]:
cost_df['Cost'].notna().sum()

46

Use the below to check data availability for specific systems

In [9]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [10]:
# TODO move to function

In [11]:
cost_df['Publication date']

39     2023-12-06
46     2023-11-30
54     2023-11-22
59     2023-11-15
83     2023-11-02
          ...    
1095   2016-09-26
1131   2016-01-27
1135   2015-12-10
1137   2015-12-08
1141   2015-10-01
Name: Publication date, Length: 148, dtype: datetime64[ns]

In [12]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [13]:
cost_df['Publication date']

39      2023-12-06 00:00:00
46      2023-11-30 00:00:00
54      2023-11-22 00:00:00
59      2023-11-15 00:00:00
83      2023-11-02 00:00:00
               ...         
1095    2016-09-26 00:00:00
1131    2016-01-27 00:00:00
1135    2015-12-10 00:00:00
1137    2015-12-08 00:00:00
1141    2015-10-01 00:00:00
Name: Publication date, Length: 148, dtype: object

In [14]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [15]:
cost_df['Cost (inflation-adjusted)'].dropna()

54      1.981978e+08
59      1.411296e+07
135     7.195242e+07
174     2.578749e+07
186     1.348447e+07
188     3.463642e+06
225     1.326337e+07
256     7.115561e+06
263     4.411429e+08
264     5.336671e+06
279     2.851964e+07
329     3.348068e+06
334     1.004420e+07
340     4.039826e+06
374     2.905966e+07
375     1.367547e+07
376     1.795109e+06
380     9.361108e+06
439     2.667208e+07
443     3.578290e+06
472     1.732684e+06
495     3.632424e+06
514     9.221837e+06
542     1.185057e+08
549     1.455102e+07
573     7.106392e+06
582     1.780796e+06
593     9.497581e+06
608     1.820094e+06
630     1.080890e+07
646     7.231038e+05
655     1.906140e+07
656     4.765351e+06
724     2.287597e+08
759     5.851089e+06
769     1.220058e+06
810     1.190141e+07
835     2.401803e+07
922     1.545774e+05
932     2.475479e+06
982     1.618421e+06
1041    1.358260e+05
1083    3.490964e+04
1088    8.727411e+03
1094    1.647856e+05
1095    2.636569e+05
Name: Cost (inflation-adjusted), d

In [16]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

46

# Regression

In [17]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [18]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     43.63
Date:                Thu, 18 Apr 2024   Prob (F-statistic):           4.34e-08
Time:                        12:31:20   Log-Likelihood:                -46.234
No. Observations:                  46   AIC:                             96.47
Df Residuals:                      44   BIC:                             100.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -632.9014     96.835     -6.536      0.000    -828.060    -437.743
x1             0.3164      0.048      6.605      0.000       0.220       0.413
==============================================================================
Omnibus:                        4.371   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.112   Jarque-Bera (JB):                3.342
Skew:                           0.635   Prob(JB):                        0.188
Kurtosis:                       3.362   Cond. No.                     1.96e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.96e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
print_growth_rates(reg_results)

0.32 OOMs/year (95% CI: 0.22, 0.41)
2.1x/year (95% CI: 1.7x, 2.6x)
doubling time of 11 months (95% CI: 9, 16)


In [20]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [21]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,4.716044,0.319061,4.073018,5.359070,3.209611,6.222477,2015.00000,2015-01-01
1,4.748007,0.314468,4.114239,5.381776,3.245503,6.250512,2015.10101,2015-02-06
2,4.779970,0.309882,4.155444,5.404497,3.281341,6.278600,2015.20202,2015-03-15
3,4.811934,0.305304,4.196633,5.427234,3.317125,6.306742,2015.30303,2015-04-21
4,4.843897,0.300734,4.237806,5.449987,3.352856,6.334938,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.752546,0.185637,7.378420,8.126672,6.339808,9.165284,2024.59596,2024-08-06
96,7.784509,0.189737,7.402120,8.166899,6.369561,9.199458,2024.69697,2024-09-12
97,7.816472,0.193871,7.425751,8.207194,6.399249,9.233695,2024.79798,2024-10-19
98,7.848435,0.198037,7.449317,8.247554,6.428875,9.267996,2024.89899,2024-11-25


# Plots

In [22]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
if estimation_method == 'amortized':
    title_text = 'Amortized cost of hardware to train large ML systems'
else:
    title_text = 'Cost of cloud compute to train large ML systems'
fig.update_layout(title_text=title_text)

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [23]:
label_systems = ['GNMT', 'Megatron-BERT', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

fig = px.scatter(
    cost_df.loc[cost_df['System'].isin(label_systems)],
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)

fig.add_scatter(
    x=cost_df['Publication date'],
    y=cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
if estimation_method == 'amortized':
    title_text = 'Amortized cost of hardware to train large ML systems'
else:
    title_text = 'Cost of cloud compute to train large ML systems'
fig.update_layout(title_text=title_text)

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [24]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation",Permissive license (depr.),NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,API access,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,197734880.0,1.981978e+08
59,Nemotron-3-8B,Language,"Chat,Language generation",Permissive license (depr.),NVIDIA AI Foundation Models: Build Custom Ente...,2023-11-15 00:00:00,NVIDIA,8.000000e+09,1.800000e+23,NaN,...,NVIDIA A100,United States of America,NaN,NaN,1024.0,0.34,NaN,NaN,14080000.0,1.411296e+07
83,Yi-34B,Language,Chat,Permissive license (depr.),NaN,2023-11-02 00:00:00,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,360000000.0,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,241440.0,2.636569e+05
1131,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,29400000.0,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10 00:00:00,Microsoft,6.000000e+07,1.210000e+19,1280000.0,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,163339200.0,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN


In [25]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)